In [1]:
import sys
import numpy as np
import random
import pandas as pd
sys.path.append('dna2vec/dna2vec/dna2vec/')

In [2]:
from multi_k_model import MultiKModel

In [3]:
import sys
print(sys.version)

3.6.7 |Anaconda custom (64-bit)| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]


In [4]:
def getSequenceEmbeddingRandomKMer(model,sequence,embeddingLenght):
    currentSequence = sequence.replace("N","")    
    embeddingVector = [0.0]*embeddingLenght
    while(currentSequence):     
        limit = min(8,len(currentSequence))
        kmer = random.randint(3,limit,random_state=21)
        evaluateSequence = currentSequence[:kmer]
        embeddingVector += model.vector(evaluateSequence)
        currentSequence = currentSequence[kmer:]        
    return embeddingVector        

In [5]:
def getSequenceEmbeddingWithKMer(model,sequence,kmer,embeddingLenght):
    currentSequence = sequence.replace("N","")    
    embeddingVector = [0.0]*embeddingLenght
    while(currentSequence):             
        chosen_kmer = min(len(currentSequence),kmer)   
        if((len(currentSequence) - kmer) < 3 and (len(currentSequence) - kmer)>0):
            chosen_kmer = chosen_kmer -  (3-(len(currentSequence) - kmer))   
        evaluateSequence = currentSequence[:chosen_kmer]
        embeddingVector += model.vector(evaluateSequence)
        currentSequence = currentSequence[chosen_kmer:]            
    return embeddingVector      

In [6]:
def getSequenceEmbeddingWithKMerLow(model,sequence,kmer,embeddingLenght):
    currentSequence = sequence.replace("N","")    
    embeddingVector = [0.0]*embeddingLenght
    while(currentSequence):             
        chosen_kmer = min(len(currentSequence),kmer)   
        if((len(currentSequence) - kmer) < 3 and (len(currentSequence) - kmer)>0):
            chosen_kmer = len(currentSequence)   
        evaluateSequence = currentSequence[:chosen_kmer]
        embeddingVector += model.vector(evaluateSequence)
        currentSequence = currentSequence[chosen_kmer:]            
    return embeddingVector      

In [7]:
train_filepath = "train_workspace/"
filepath_model = "dna_complete/"
animalList = ["Human","Pig","Chicken","Rat","Mouse","Dog"]
levelList = ["Strict","Intermediate","Relaxed"]
mutation_levels = ["VeryLow","Low","Medium","High"]

In [8]:
def get_df_dict(level):
    df_dict = {}
    df_dict["VeryLow"] = {}
    df_dict["Low"] = {}
    df_dict["Medium"] = {}
    df_dict["High"] = {}
    current_file_path = train_filepath + level + "/mutated/ohnologs-complete-"    
    for mutation_level in mutation_levels:                
        df_dict[mutation_level] = pd.read_pickle(current_file_path + mutation_level + ".pkl")        
    return df_dict

In [9]:
kmer = 8

In [10]:
models_dict = {}
models_dict["Strict"] = {}
models_dict["Intermediate"] = {}
models_dict["Relaxed"] = {}
for level in levelList:
    for animal in animalList:
        model_file_path = filepath_model + "dna_" + animal.lower() + "/results/" + animal.lower() + "_model_cdna.w2v"
        models_dict[level][animal] = MultiKModel(model_file_path)    

In [11]:
len(models_dict["Intermediate"]["Human"].vector("AAA"))

100

In [20]:
df_embedding_dict = {}
df_embedding_dict_Mutated = {}
for level in levelList:
    df_dict = get_df_dict(level)    
    df_embedding_dict[level] = {}
    df_embedding_dict_Mutated[level] = {}
    for mutation_level in mutation_levels:                    
        embeddings_sequence_1 = []
        embeddings_sequence_2 = []        
        
        embeddings_sequence_1_mutated = []
        embeddings_sequence_2_mutated = []    
        for index, row in df_dict[mutation_level].iterrows():
            model = models_dict[level][row["Species"]]
            embeddings_sequence_1.append(getSequenceEmbeddingWithKMer(model,row["Sequence-1"],kmer,100))
            embeddings_sequence_2.append(getSequenceEmbeddingWithKMer(model,row["Sequence-2"],kmer,100))
            
            embeddings_sequence_1_mutated.append(getSequenceEmbeddingWithKMer(model,row["Sequence-1-Mutated"],kmer,100))
            embeddings_sequence_2_mutated.append(getSequenceEmbeddingWithKMer(model,row["Sequence-2-Mutated"],kmer,100))

        df_embedding_dict[level][mutation_level] = []
        df_embeddings_1 = pd.DataFrame(embeddings_sequence_1)        
        df_embeddings_1.columns = ['Embedding1_' + str(i) for i in range(0,100)]                               
        
        df_embeddings_2 = pd.DataFrame(embeddings_sequence_2)        
        df_embeddings_2.columns = ['Embedding2_' + str(i) for i in range(0,100)]                
        
        df_embedding_dict[level][mutation_level].append(df_embeddings_1)
        df_embedding_dict[level][mutation_level].append(df_embeddings_2)
        
        
        df_embedding_dict_Mutated[level][mutation_level] = []
        df_embeddings_1_mutated = pd.DataFrame(embeddings_sequence_1_mutated)        
        df_embeddings_1_mutated.columns = ['M_Embedding1_' + str(i) for i in range(0,100)]                               
        
        df_embeddings_2_mutated = pd.DataFrame(embeddings_sequence_2_mutated)        
        df_embeddings_2_mutated.columns = ['M_Embedding2_' + str(i) for i in range(0,100)]                
        
        df_embedding_dict_Mutated[level][mutation_level].append(df_embeddings_1_mutated)
        df_embedding_dict_Mutated[level][mutation_level].append(df_embeddings_2_mutated)

In [24]:
for level in levelList:    
    for mutation_level in mutation_levels:    
        df_embedding_dict[level][mutation_level][0].to_pickle(train_filepath + level + "/embeddings_dna2vec/mutated/" + "ohnologs-" + "1-vec-"+ str(kmer) + "kmer-" + mutation_level + ".pkl")
        df_embedding_dict[level][mutation_level][1].to_pickle(train_filepath + level + "/embeddings_dna2vec/mutated/" + "ohnologs-" + "2-vec-"+ str(kmer) + "kmer-" + mutation_level + ".pkl")
        df_embedding_dict_Mutated[level][mutation_level][0].to_pickle(train_filepath + level + "/embeddings_dna2vec/mutated/" + "m_ohnologs-" + "1-vec-"+ str(kmer) + "kmer-" + mutation_level + ".pkl")
        df_embedding_dict_Mutated[level][mutation_level][1].to_pickle(train_filepath + level + "/embeddings_dna2vec/mutated/" + "m_ohnologs-" + "2-vec-"+ str(kmer) + "kmer-" + mutation_level + ".pkl")

In [21]:
df_embedding_dict["Strict"]["High"][0].head()

,Embedding1_0,Embedding1_1,Embedding1_2,Embedding1_3,Embedding1_4,Embedding1_5,Embedding1_6,Embedding1_7,Embedding1_8,Embedding1_9,...,Embedding1_90,Embedding1_91,Embedding1_92,Embedding1_93,Embedding1_94,Embedding1_95,Embedding1_96,Embedding1_97,Embedding1_98,Embedding1_99
0,-44.596029,-61.756636,10.059460,36.134568,24.379887,-81.081434,48.220245,26.109957,91.498196,-0.582015,...,-7.474908,0.067471,-94.353768,-7.527687,-124.497870,15.390788,-91.922412,0.732603,0.177474,-113.270618
1,80.997234,-164.428671,11.611871,-11.995442,38.196104,-137.885649,221.601906,63.859184,134.945870,-94.679555,...,17.407122,31.330681,-251.195289,-5.860897,-221.245864,24.617813,-255.533832,-13.725622,93.337069,-202.745104
2,-16.512792,-45.387001,-1.135553,18.020396,27.439804,-46.499858,40.632111,5.028233,66.191693,-7.862134,...,11.211893,-15.428725,-80.687848,-15.306750,-95.489268,3.256722,-77.313524,-2.780348,-0.843615,-85.069809
3,-1.512923,-17.456491,2.355848,13.005734,4.113096,-2.212156,14.677081,3.845454,7.374721,-7.362568,...,2.738486,-9.991025,-16.074188,-16.206103,-19.082065,4.342779,-15.240538,-10.324697,-0.581034,-18.723295
4,-35.541973,-14.488550,12.987731,12.169984,38.354464,-21.585262,46.300773,-2.479553,29.373251,-12.000128,...,33.314357,12.886476,-78.575210,-72.100819,-73.401387,-30.356501,-98.667253,42.129240,10.630233,-79.750837


In [22]:
df_embedding_dict_Mutated["Strict"]["High"][0].head()

,M_Embedding1_0,M_Embedding1_1,M_Embedding1_2,M_Embedding1_3,M_Embedding1_4,M_Embedding1_5,M_Embedding1_6,M_Embedding1_7,M_Embedding1_8,M_Embedding1_9,...,M_Embedding1_90,M_Embedding1_91,M_Embedding1_92,M_Embedding1_93,M_Embedding1_94,M_Embedding1_95,M_Embedding1_96,M_Embedding1_97,M_Embedding1_98,M_Embedding1_99
0,-39.472377,-44.192033,9.964507,31.088967,24.663256,-68.922827,43.415773,27.335079,80.407817,-7.771471,...,-7.528718,-0.024174,-73.955010,-3.313923,-105.237071,13.568430,-79.195485,1.887270,-2.746446,-92.302292
1,64.836815,-144.695733,18.905258,0.423586,30.639172,-114.272871,189.066898,53.640197,130.118663,-82.879873,...,-0.463426,10.666150,-198.573426,-8.572368,-192.473512,17.223968,-239.440352,-5.877865,79.285015,-197.588224
2,-18.555755,-37.171268,13.446967,5.479718,25.688694,-36.043900,36.470430,15.809576,59.245241,-5.827751,...,2.505122,-6.070827,-65.089789,-11.878075,-78.171705,1.387310,-69.821944,-3.820156,-8.789678,-75.725951
3,1.704613,-11.463163,0.676982,16.373105,1.747263,-1.163784,9.217650,6.698717,7.678625,-6.162662,...,2.677826,-8.504213,-14.296372,-14.559813,-7.767167,5.855014,-15.532859,-5.366099,1.248430,-16.244287
4,-36.389966,-28.036366,2.022434,20.985730,29.062955,-26.671728,40.044195,-3.796868,31.395068,-19.266860,...,33.951392,9.376654,-79.814748,-70.836473,-71.008435,-23.943026,-97.242807,39.161855,9.391607,-76.678009
